In [3]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [41]:
import requests

cookies = {
    '2A21_92c8_saltkey': 'S1mX69z6',
    '2A21_92c8_lastvisit': '1684768064',
    '__utmz': '30105572.1684772353.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none)',
    '__utma': '30105572.135848573.1684772353.1684772353.1684850662.2',
    '__utmc': '30105572',
    '__utmt': '1',
    '2A21_92c8_sid': '66Ah1z',
    '2A21_92c8_viewid': 'tid_524607',
    '2A21_92c8_visitedfid': '715D369D889D341',
    '2A21_92c8_forum_lastvisit': 'D_905_1684772352D_457_1684850685D_341_1684850689D_889_1684850710D_369_1684850714D_715_1684851023',
    '2A21_92c8_lastact': '1684851023%09home.php%09misc',
    '2A21_92c8_sendmail': '1',
    '__utmb': '30105572.10.10.1684850662',
}

headers = {
    'authority': 'www.jbtalks.cc',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    # 'cookie': '2A21_92c8_saltkey=S1mX69z6; 2A21_92c8_lastvisit=1684768064; __utmz=30105572.1684772353.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utma=30105572.135848573.1684772353.1684772353.1684850662.2; __utmc=30105572; __utmt=1; 2A21_92c8_sid=66Ah1z; 2A21_92c8_viewid=tid_524607; 2A21_92c8_visitedfid=715D369D889D341; 2A21_92c8_forum_lastvisit=D_905_1684772352D_457_1684850685D_341_1684850689D_889_1684850710D_369_1684850714D_715_1684851023; 2A21_92c8_lastact=1684851023%09home.php%09misc; 2A21_92c8_sendmail=1; __utmb=30105572.10.10.1684850662',
    'pragma': 'no-cache',
    'referer': 'https://www.jbtalks.cc/forum-905-1.html',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
}

response = requests.get('https://www.jbtalks.cc/forum.php', cookies=cookies, headers=headers)

In [42]:
soup = BeautifulSoup(response.content, "lxml")

In [43]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]
subtopics = [a_ for a_ in a if a_.startswith('forum-') and a_.endswith('.html')]
subtopics = sorted(list(set(subtopics)))

with open('subtopics.json', 'w') as fopen:
    json.dump(subtopics, fopen)

In [58]:
def get_url(url):
    while True:
        try:
            response = requests.get(url, cookies=cookies, headers=headers)
            soup = BeautifulSoup(response.content, "lxml")
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)
    try:
        span = soup.find_all('span')
        max_page = [s.get('title') for s in span if '页' in s.get('title', '') and '共' in s.get('title', '')][0]
        max_page = int(max_page.split()[1])
    except:
        max_page = -1
    
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_ for a_ in a if a_.startswith('thread-') and a_.endswith('.html')]
    return a, max_page

In [47]:
url = f'https://www.jbtalks.cc/{s}'
url.split('-')

['https://www.jbtalks.cc/forum', '100', '1.html']

In [53]:
aranged = [2,3,4,5]
urls = []
for a in aranged:
    urls.append('-'.join(url.split('-')[:-1]) + f'-{a}.html')
urls

['https://www.jbtalks.cc/forum-100-2.html',
 'https://www.jbtalks.cc/forum-100-3.html',
 'https://www.jbtalks.cc/forum-100-4.html',
 'https://www.jbtalks.cc/forum-100-5.html']

In [59]:
# !rm -rf topics
!mkdir topics

mkdir: cannot create directory ‘topics’: File exists


In [60]:
max_worker = 20

for i in tqdm(range(len(subtopics))):
    a_ = subtopics[i]
    filename = os.path.join('topics', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    base_url = f'https://www.jbtalks.cc/{a_}'
    
    filtered, max_page = get_url(base_url)
    
    for i in tqdm(range(2, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = []
        for a in aranged:
            urls.append('-'.join(url.split('-')[:-1]) + f'-{a}.html')

        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(get_url, url): url for url in urls}

            for future in as_completed(futures):
                filtered.extend(future.result()[0])
                
    filtered = list(set(filtered))
    with open(filename, 'w') as fopen:
        json.dump(filtered, fopen)

 42%|████████████████▋                       | 205/493 [00:01<00:02, 140.49it/s]
0it [00:00, ?it/s]

100%|█████████████████████████████████████████████| 3/3 [00:03<00:00,  1.16s/it]

0it [00:00, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.20s/it]

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.23s/it]

100%|█████████████████████████████████████████████| 2/2 [00:02<00:00,  1.19s/it]

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.24s/it]

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.15s/it]

100%|█████████████████████████████████████████████| 6/6 [00:06<00:00,  1.17s/it]

0it [00:00, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.37s/it]

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.17s/it]

0it [00:00, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.22s/it]

100%|██████████████

 47%|███████████████████▍                     | 234/493 [01:20<05:45,  1.33s/it]
0it [00:00, ?it/s]
 50%|████████████████████▌                    | 248/493 [02:01<23:24,  5.73s/it]
0it [00:00, ?it/s]
 51%|████████████████████▊                    | 251/493 [02:04<11:02,  2.74s/it]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.21s/it]


 55%|██████████████████████▋                  | 273/493 [02:48<06:48,  1.86s/it]
0it [00:00, ?it/s]
 57%|███████████████████████▎                 | 281/493 [03:06<07:12,  2.04s/it]
0it [00:00, ?it/s]
  0%|                                                     | 0/1 [00:00<?, ?it/s]


 61%|████████████████████████▉                | 300/493 [03:35<05:09,  1.60s/it]
0it [00:00, ?it/s]
 61%|█████████████████████████                | 301/493 [03:35<03:45,  1.17s/it]
0it [00:00, ?it/s]
 61%|█████████████████████████                | 302/493 [03:36<02:46,  1.14it/s]
0it [00:00, ?it/s]
 61%|█████████████████████████▏               | 303/493 [03:36<02:07,  1.49it/s]
0it [00:00, ?it/s]
 62%|█████████████████████████▎               | 304/493 [03:36<01:39,  1.91it/s]
0it [00:00, ?it/s]
 62%|█████████████████████████▎               | 305/493 [03:36<01:21,  2.30it/s]
0it [00:00, ?it/s]
 62%|█████████████████████████▍               | 306/493 [03:36<01:07,  2.79it/s]
0it [00:00, ?it/s]
 62%|█████████████████████████▌               | 307/493 [03:37<00:57,  3.25it/s]
0it [00:00, ?it/s]
 62%|█████████████████████████▌               | 308/493 [03:37<00:49,  3.74it/s]
0it [00:00, ?it/s]
 63%|█████████████████████████▋               | 309/493 [03:37<00:48,  3.81it/s]
0it [00:00, ?it/s]


 67%|███████████████████████████▌             | 332/493 [04:09<05:55,  2.21s/it]
0it [00:00, ?it/s]
 69%|████████████████████████████▏            | 339/493 [04:27<06:57,  2.71s/it]
0it [00:00, ?it/s]
 71%|█████████████████████████████            | 349/493 [04:44<04:29,  1.87s/it]
0it [00:00, ?it/s]
 72%|█████████████████████████████▋           | 357/493 [04:54<03:15,  1.44s/it]
0it [00:00, ?it/s]
 73%|█████████████████████████████▊           | 358/493 [04:55<02:25,  1.08s/it]
0it [00:00, ?it/s]
 50%|██████████████████████▌                      | 1/2 [00:01<00:01,  1.17s/it]


...

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))



 84%|██████████████████████████████████▌      | 416/493 [07:07<06:11,  4.82s/it]
0it [00:00, ?it/s]
 85%|██████████████████████████████████▉      | 420/493 [07:13<02:59,  2.46s/it]
0it [00:00, ?it/s]
 85%|███████████████████████████████████      | 421/493 [07:14<02:15,  1.88s/it]
0it [00:00, ?it/s]
 86%|███████████████████████████████████      | 422/493 [07:14<01:38,  1.38s/it]
0it [00:00, ?it/s]
 86%|███████████████████████████████████▎     | 425/493 [07:17<01:25,  1.25s/it]
0it [00:00, ?it/s]
 87%|███████████████████████████████████▌     | 428/493 [07:21<01:35,  1.47s/it]
0it [00:00, ?it/s]
 87%|███████████████████████████████████▋     | 429/493 [07:21<01:09,  1.08s/it]
0it [00:00, ?it/s]
  0%|                                                     | 0/1 [00:00<?, ?it/s]

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))



 88%|███████████████████████████████████▉     | 432/493 [07:36<04:30,  4.43s/it]
0it [00:00, ?it/s]
 88%|████████████████████████████████████     | 434/493 [07:38<02:36,  2.65s/it]
0it [00:00, ?it/s]
 88%|████████████████████████████████████▏    | 435/493 [07:38<01:51,  1.92s/it]
0it [00:00, ?it/s]
 90%|████████████████████████████████████▊    | 442/493 [07:48<01:22,  1.62s/it]
0it [00:00, ?it/s]
 90%|████████████████████████████████████▊    | 443/493 [07:48<01:00,  1.20s/it]
0it [00:00, ?it/s]
 91%|█████████████████████████████████████▏   | 447/493 [07:53<00:57,  1.25s/it]
0it [00:00, ?it/s]
 92%|█████████████████████████████████████▊   | 454/493 [08:01<00:53,  1.38s/it]
0it [00:00, ?it/s]
 92%|█████████████████████████████████████▊   | 455/493 [08:02<00:39,  1.03s/it]
0it [00:00, ?it/s]
 92%|█████████████████████████████████████▉   | 456/493 [08:02<00:28,  1.29it/s]
0it [00:00, ?it/s]
 93%|██████████████████████████████████████   | 457/493 [08:02<00:21,  1.68it/s]
0it [00:00, ?it/s]

 98%|████████████████████████████████████████ | 481/493 [08:32<00:39,  3.33s/it]
0it [00:00, ?it/s]
 98%|████████████████████████████████████████ | 482/493 [08:32<00:26,  2.39s/it]
0it [00:00, ?it/s]
 98%|████████████████████████████████████████▏| 483/493 [08:33<00:17,  1.74s/it]
0it [00:00, ?it/s]
 99%|████████████████████████████████████████▍| 486/493 [08:41<00:16,  2.39s/it]
0it [00:00, ?it/s]
 99%|████████████████████████████████████████▌| 487/493 [08:41<00:10,  1.73s/it]
0it [00:00, ?it/s]
100%|████████████████████████████████████████▉| 492/493 [08:49<00:01,  1.69s/it]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 493/493 [08:49<00:00,  1.07s/it]


In [26]:
response = requests.get(f'https://www.jbtalks.cc/forum-381-1.html', cookies=cookies, headers=headers)

In [27]:
response

<Response [200]>

In [28]:
soup = BeautifulSoup(response.content, "lxml")

In [38]:
span = soup.find_all('span')
max_page = [s.get('title') for s in span if '页' in s.get('title', '') and '共' in s.get('title', '')][0]
max_page = int(max_page.split()[1])
max_page

39

In [39]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]
a = [a_ for a_ in a if a_.startswith('thread-') and a_.endswith('.html')]